<a href="https://colab.research.google.com/github/RRahul-Abrol/Starting-with-Agentic-Ai/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
from typing import Annotated
from dotenv import load_dotenv
load_dotenv()

from typing_extensions import TypedDict
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [9]:
from langchain.chat_models import init_chat_model

In [10]:
llm = init_chat_model("google_genai:gemini-2.5-flash", api_key=GOOGLE_API_KEY)

class State(TypedDict):
    messages: Annotated[list, add_messages]

def chatbot(state: State) -> State:
    return {"messages": [llm.invoke(state["messages"])]}

builder = StateGraph(State)
builder.add_node("chatbot_node", chatbot)

builder.add_edge(START, "chatbot_node")
builder.add_edge("chatbot_node", END)

graph = builder.compile()

In [11]:
message = {"role": "user", "content": "Who walked on the moon for the first time? Print only the name"}
# message = {"role": "user", "content": "What is the latest price of MSFT stock?"}
response = graph.invoke({"messages":[message]})

response["messages"]

[HumanMessage(content='Who walked on the moon for the first time? Print only the name', additional_kwargs={}, response_metadata={}, id='52074aa2-7298-45b7-9e78-9c141f14e218'),
 AIMessage(content='Neil Armstrong', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b5ab8-633e-7740-8954-f9dcc1eee105-0', usage_metadata={'input_tokens': 15, 'output_tokens': 51, 'total_tokens': 66, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 49}})]

In [12]:
state = None
while True:
    in_message = input("You: ")
    if in_message.lower() in {"quit","exit"}:
        break
    if state is None:
        state: State = {
            "messages": [{"role": "user", "content": in_message}]
        }
    else:
        state["messages"].append({"role": "user", "content": in_message})

    state = graph.invoke(state)
    print("Bot:", state["messages"][-1].content)

You: hi
Bot: Hi there! How can I help you today?
You: prime minister
Bot: A **prime minister** is typically the **head of government** in a **parliamentary system**.

They are responsible for leading the executive branch of government and are usually the leader of the political party or coalition with the most seats in the legislature (parliament).

To give you more specific information, could you tell me:

*   **Which country's prime minister** are you interested in?
*   **What kind of information** are you looking for? (e.g., current, historical, their role, recent news, etc.)
You: india
Bot: The current Prime Minister of India is **Narendra Modi**.

Here are some key facts about him:

*   **Current PM:** He assumed office on May 26, 2014, and is currently serving his second consecutive term.
*   **Political Party:** He is a member of the **Bharatiya Janata Party (BJP)**.
*   **Role:** In India, the Prime Minister is the head of the Union government, the chief advisor to the Presiden